In [1]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
import matplotlib.pyplot as plt

In [2]:
# Transformations that are used in the dataset
trans = transforms.Compose([
    
    transforms.ToTensor()])

In [3]:
# Loading the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./', train= True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root='./', train =False, transform=trans)


In [4]:
# We are splitting the training set into a training set and a validation set
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [5]:
# Hyperparameters for the model
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [6]:
# Creating dataloaders for the respective datasets 
train_loader = DataLoader ( dataset=  train_dataset, batch_size = batch_size, shuffle= True)
valid_loader = DataLoader(dataset =val_dataset, batch_size = batch_size,   shuffle =False )
test_loader = DataLoader ( dataset = test_dataset, batch_size= batch_size, shuffle = False)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(7 * 7 * 64, 1000),
            nn.Linear(1000, 10)
        )

    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out, 1) 
        out = self.classifier(out)
        return out

model = CNN().to(device)


In [9]:
loss_funcn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_step = len(train_loader)
loss_list = []
total = 0  
correct = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Doing the forward pass
        outputs = model(images)
        loss = loss_funcn(outputs, labels)
        loss_list.append(loss.item())
        
        # back propagation

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Computing accuracy
        total += labels.size(0)
        predicted = torch.max(outputs.data, 1)[1]
        correct += (predicted == labels).sum().item()

    # Calculate accuracy for the whole dataset
    accuracy = correct / total * 100

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch + 1, num_epochs,
                                                                   loss.item(), accuracy))

print('Accuracy on the whole dataset: {:.2f}%'.format(accuracy))


Epoch [1/10], Loss: 0.0642, Accuracy: 94.68%
Epoch [2/10], Loss: 0.3518, Accuracy: 96.29%
Epoch [3/10], Loss: 0.1144, Accuracy: 96.97%
Epoch [4/10], Loss: 0.0049, Accuracy: 97.39%
Epoch [5/10], Loss: 0.0096, Accuracy: 97.66%
Epoch [6/10], Loss: 0.0189, Accuracy: 97.86%
Epoch [7/10], Loss: 0.0918, Accuracy: 98.02%
Epoch [8/10], Loss: 0.0392, Accuracy: 98.14%
Epoch [9/10], Loss: 0.0290, Accuracy: 98.25%
Epoch [10/10], Loss: 0.0351, Accuracy: 98.34%
Accuracy on the whole dataset: 98.34%


In [11]:
# Evaluating the model on the testing set


model.eval()
testing_labels= []
testing_predictions =[]

with torch.no_grad():
    for images, labels in test_loader:
        images= images.to(device)
        labels =labels.to(device )

        outputs = model(images )
        predicted=  torch.max( outputs.data, 1 )[1] 

        testing_labels.extend(labels.cpu().numpy())
        testing_predictions.extend(predicted.cpu().numpy())


testing_accuracy= (sum([1 for i, j in zip( testing_labels, testing_predictions) if i == j]) / len( testing_labels)) * 100

testing_confusion_matrix= confusion_matrix( testing_labels, testing_predictions)

print('Test Accuracy: {:.2f}%'.format( testing_accuracy ))
print(testing_confusion_matrix )


Test Accuracy: 99.04%
[[ 979    0    0    0    0    0    0    1    0    0]
 [   0 1128    1    2    0    0    0    2    1    1]
 [   1    0 1025    0    0    0    0    6    0    0]
 [   0    0    0 1007    0    3    0    0    0    0]
 [   0    1    0    0  960    0    1    1    4   15]
 [   1    0    0    2    0  887    1    0    0    1]
 [  11    2    0    1    2    3  937    0    2    0]
 [   0    2    1    0    0    0    0 1024    1    0]
 [   4    0    2    1    0    2    0    1  961    3]
 [   1    0    1    0    2    3    0    4    2  996]]


In [12]:
_PATH = "C:\\Users\\SWETHA"

In [13]:
# Saving the model
torch.save(model.state_dict(), _PATH + '\\conv_net_model.ckpt')